<a href="https://colab.research.google.com/github/nivash-T/Bag-of-words-model/blob/main/flat_type_ann_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


In [ ]:

tf.__version__

'2.18.0'

In [ ]:
data = pd.read_csv(r'D:/flux_data/flat_type_merged_output.csv')


In [ ]:
X=data.iloc[:,0:5]
print(X)

     flat_type_w0  flat_type_w3  flat_type_h0  flat_type_h1  flat_type_h2
0           0.032        0.0076        0.0035        0.0064        0.0028
1           0.032        0.0133        0.0041        0.0058        0.0026
2           0.027        0.0119        0.0049        0.0052        0.0020
3           0.037        0.0034        0.0055        0.0070        0.0026
4           0.037        0.0156        0.0033        0.0058        0.0016
..            ...           ...           ...           ...           ...
495         0.017        0.0075        0.0033        0.0062        0.0026
496         0.012        0.0184        0.0041        0.0044        0.0010
497         0.037        0.0132        0.0053        0.0040        0.0010
498         0.032        0.0066        0.0019        0.0056        0.0028
499         0.032        0.0013        0.0035        0.0076        0.0016

[500 rows x 5 columns]


In [ ]:
Y=data.iloc[:,5:]
print(Y)

          0.0  0.0031166593785613  0.0062333187571226  0.0093499781356839  \
0    0.342918            0.337754            0.334502            0.329246   
1    0.275163            0.266829            0.259708            0.250296   
2    0.186589            0.182058            0.178432            0.173419   
3    0.325997            0.322078            0.319638            0.315422   
4    0.359481            0.340322            0.322304            0.299191   
..        ...                 ...                 ...                 ...   
495  0.125590            0.124520            0.124052            0.122850   
496  0.016982            0.015299            0.013653            0.011864   
497  0.283761            0.271587            0.261064            0.248362   
498  0.397174            0.392261            0.389489            0.384676   
499  0.352841            0.349550            0.348059            0.344823   

     0.0124666375142452  0.0155832968928065  0.0186999562713678  \
0       

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)





print(f"Training set size: {len(X_train)}")

print(f"Test set size: {len(X_test)}")



Training set size: 400
Test set size: 100


In [ ]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_train=np.array(X_train)


X_test = sc.transform(X_test)



X_test = np.array(X_test)


In [ ]:
mu = sc.mean_  # Mean of each feature before scaling
sigma = sc.scale_  # Standard deviation of each feature before scaling

print("Mean (mu) of original data:", mu)
print("Standard Deviation (sigma) of original data:", sigma)

Mean (mu) of original data: [0.0245875  0.01003125 0.0030535  0.0059525  0.001931  ]
Standard Deviation (sigma) of original data: [0.0087674  0.00535615 0.00159309 0.00115241 0.00056039]


In [ ]:
# Initialize scaler
y_scaler = StandardScaler()

# Fit on training data and transform
Y_train = y_scaler.fit_transform(Y_train)
Y_train = np.array(Y_train)

# Transform test data using the same scaler
Y_test = y_scaler.transform(Y_test)
Y_test = np.array(Y_test)

In [ ]:
import joblib
joblib.dump(sc, 'D:/flux_models/flat_type/x_scaler.pkl')
joblib.dump(y_scaler, 'D:/flux_models/flat_type/y_scaler.pkl')

['D:/flux_models/flat_type/y_scaler.pkl']

In [ ]:
import tensorflow as tf

In [ ]:
import os
model_path="D:/flux_models/flat_type/flat_type_model.keras"
if os.path.exists(model_path):
    print("Model found! Loading the model...")
    ann = tf.keras.models.load_model(model_path)
else:
    print("Model not found. Training a new model...")
    ann = None

Model found! Loading the model...


In [ ]:
if not ann:
    # Building the ANN
    ann = tf.keras.models.Sequential()

    # Adding input layer and the first hidden layer with 100 neurons
    ann.add(tf.keras.layers.Dense(units=100, activation='relu', input_dim=5))  # Input layer with 5 features

    # Adding the second hidden layer with 100 neurons
    ann.add(tf.keras.layers.Dense(units=100, activation='relu'))

    # Adding output layer with 2016 neurons
    ann.add(tf.keras.layers.Dense(units=2016, activation='linear'))
    # Compile the ANN for a multi-output regression task
    ann.compile(optimizer='adam',
                loss='mean_squared_error',  # Use mean squared error for regression
                metrics=['mean_absolute_error'])  # Additional metric for evaluation


In [ ]:
# Display the model summary
ann.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 100)            │           600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2016)           │       203,616 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 642,950 (2.45 MB)

 Trainable params: 214,316 (837.17 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 428,634 (1.64 MB)

In [ ]:


# Train the ANN
history = ann.fit(X_train, Y_train,
                  batch_size=32,  # Batch size
                  epochs=1000,  # Number of epochs
                  validation_split=0.2,  # Use 20% of training data for validation
                  verbose=1)  # Display training progress


Epoch 1/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.6554 - mean_absolute_error: 0.6559 - val_loss: 0.3165 - val_mean_absolute_error: 0.4263
Epoch 2/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2211 - mean_absolute_error: 0.3485 - val_loss: 0.1062 - val_mean_absolute_error: 0.2253
Epoch 3/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0934 - mean_absolute_error: 0.1935 - val_loss: 0.0736 - val_mean_absolute_error: 0.1885
Epoch 4/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0666 - mean_absolute_error: 0.1551 - val_loss: 0.0477 - val_mean_absolute_error: 0.1282
Epoch 5/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0426 - mean_absolute_error: 0.1121 - val_loss: 0.0412 - val_mean_absolute_error: 0.1218
Epoch 6/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0477 - mean_absolute_error: 0.1085 - val_loss: 0.0353 - val_mean_absolute_error: 0.1041
Epoch 7/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0456 - mean_absolute_error: 0.0

In [ ]:
ann.save(model_path)

In [ ]:
# Evaluate the ANN model on the test dataset
test_loss, test_mae = ann.evaluate(X_test, Y_test)

# Print the evaluation results
print(f"Test Loss (Mean Squared Error): {test_loss}")
print(f"Test Mean Absolute Error (MAE): {test_mae}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0198 - mean_absolute_error: 0.0553
Test Loss (Mean Squared Error): 0.018128054216504097
Test Mean Absolute Error (MAE): 0.054756056517362595


In [ ]:
from sklearn.metrics import r2_score

# Predict on the test data
Y_pred = ann.predict(X_test)

# Calculate R^2 (Coefficient of Determination)
r2 = r2_score(Y_test, Y_pred)
print(f"R^2 score: {r2}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
R^2 score: 0.9745932298100866


In [ ]:
Y_test = pd.DataFrame(Y_test)
# Calculate min, max, and mean for each row
row_stats = pd.DataFrame({
    'Min': Y_test.min(axis=1),
    'Max': Y_test.max(axis=1),
    'Mean': Y_test.mean(axis=1),
    'Min Column Index': Y_test.values.argmin(axis=1),
    'Max Column Index': Y_test.values.argmax(axis=1)
})


In [ ]:

row_stats

,Min,Max,Mean,Min Column Index,Max Column Index
0,-0.954838,0.866031,0.337377,67,145
1,0.312466,4.105141,0.822022,167,41
2,-0.730829,1.987021,1.267090,23,95
3,-0.987264,0.717600,0.226819,68,145
4,-0.990489,0.834166,0.293997,15,113
...,...,...,...,...,...
95,-0.935216,1.276782,0.648147,16,145
96,-0.784322,0.110559,-0.127562,65,110
97,-1.302506,0.319018,-0.983101,154,43
98,-0.737665,-0.008054,-0.550599,155,42


In [ ]:
df = pd.DataFrame(Y_pred)

pred_row_stats = pd.DataFrame({
    'Min': df.min(axis=1),
    'Max': df.max(axis=1),
    'Mean': df.mean(axis=1),
    'Min Column Index': df.values.argmin(axis=1),
    'Max Column Index': df.values.argmax(axis=1)
})

In [ ]:
Y_test.iloc[:,27]

0    -0.648511
1     1.744456
2    -0.615866
3    -0.642160
4    -0.643155
        ...   
95   -0.651090
96   -0.612475
97   -0.173942
98   -0.196050
99   -0.651614
Name: 27, Length: 100, dtype: float64

In [ ]:
df[27]

0    -0.626669
1     1.527014
2    -0.710657
3    -0.639186
4    -0.646996
        ...   
95   -0.772747
96   -0.618124
97   -0.083155
98   -0.326557
99   -0.579780
Name: 27, Length: 100, dtype: float32

In [ ]:
Y_test.iloc[:,168]

0     0.849004
1     0.312510
2     1.587729
3     0.702281
4     0.777532
        ...   
95    1.255197
96    0.048034
97   -1.292832
98   -0.735280
99    2.323289
Name: 168, Length: 100, dtype: float64

In [ ]:
df[168]

0     0.862438
1     0.318708
2     1.610726
3     0.702459
4     0.712948
        ...   
95    1.284748
96    0.196966
97   -1.287943
98   -0.701009
99    2.333936
Name: 168, Length: 100, dtype: float32

In [ ]:
pred_row_stats

,Min,Max,Mean,Min Column Index,Max Column Index
0,-0.899465,0.894592,0.341860,1073,1389
1,0.318648,2.043861,0.767377,421,43
2,-1.052780,1.935708,1.254382,797,1368
3,-1.001029,0.717703,0.222920,16,949
4,-0.904728,0.839534,0.251346,1777,478
...,...,...,...,...,...
95,-1.046969,1.316000,0.671616,822,1657
96,-0.894780,0.298883,-0.047997,521,1738
97,-1.985262,2.077610,-0.888913,1050,1048
98,-0.705006,-0.082511,-0.537455,1476,62
